## AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

Facebook kısa süre önce mevcut "maximum bidding" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan "average bidding"’i tanıttı.

Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test etmeye karar verdi ve average bidding'in maximum bidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.


A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor. Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average Bidding uygulanmıştır.

#### Görev 1: Veriyi Hazırlama ve Analiz Etme

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind

In [4]:
# Verilerimizi okutuyoruz. Ancak Test ve Control grubunu ayrı ayrı atıyoruz; 
dataframe_control = pd.read_excel("dataset/ab_testing.xlsx" , sheet_name="Control Group")
dataframe_test = pd.read_excel("dataset/ab_testing.xlsx" , sheet_name="Test Group")

In [5]:
dataframe_control.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [6]:
dataframe_test.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [11]:
# Bu iki grup için analizlere göz atmak istiyoruz. Ancak hepsine df.shape, df.head şeklinde tek tek yazmak zor. Bu sebeple bir fonksiyon yazıyoruz;
def check_df(dataframe, head=5):
    print("###################SHAPE###################")
    print(dataframe.shape)
    print("###################TYPE###################")
    print(dataframe.dtypes)
    print("###################TAIL###################")
    print(dataframe.tail())
    print("###################NA###################")
    print(dataframe.isnull().sum)
    print("###################QUANTİLES###################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
check_df(dataframe_control)
#####
check_df(dataframe_test)


###################SHAPE###################
(40, 4)
###################TYPE###################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
###################TAIL###################
       Impression        Click    Purchase      Earning
35  132064.219003  3747.157544  551.072406  2256.975589
36   86409.941796  4608.256205  345.046033  1781.357690
37  123678.934234  3649.073787  476.168128  2187.721217
38  101997.494099  4736.353369  474.613537  2254.563834
39  121085.881220  4285.178608  590.406020  1289.308948
###################NA###################
<bound method NDFrame._add_numeric_operations.<locals>.sum of     Impression  Click  Purchase  Earning
0        False  False     False    False
1        False  False     False    False
2        False  False     False    False
3        False  False     False    False
4        False  False     False    False
5        False  False     False    False
6        False  False     False   

In [17]:
dataframe_control["group"] = "control"
dataframe_test["group"] = "test"
# axis 0 ile satır bazlı 1 ile stün bazlı birlieştirme yapılmaktadır. 
# ignore_index True ile birleştirme kısmında yeniden değil kaldığı yerden indexlemeye devam etmektedir. 
df = pd.concat([dataframe_control,dataframe_test], axis=0,ignore_index=True)
df


,Impression,Click,Purchase,Earning,group
0,82529.459271,6090.077317,665.211255,2311.277143,control
1,98050.451926,3382.861786,315.084895,1742.806855,control
2,82696.023549,4167.965750,458.083738,1797.827447,control
3,109914.400398,4910.882240,487.090773,1696.229178,control
4,108457.762630,5987.655811,441.034050,1543.720179,control
...,...,...,...,...,...
75,79234.911929,6002.213585,382.047116,2277.863984,test
76,130702.239410,3626.320072,449.824592,2530.841327,test
77,116481.873365,4702.782468,472.453725,2597.917632,test
78,79033.834921,4495.428177,425.359102,2595.857880,test


##### Görev 2:  A/B Testinin Hipotezinin Tanımlanması

In [18]:
# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark vardır.)

In [26]:
# iki grup arasındaki ortalamaları kıyaslamak için groupby ile satınalma ortalamalarını getirelim; 
df.groupby("group").agg({"Purchase":"mean"})

,Purchase
group,
control,550.894059
test,582.106097


In [27]:
# Yukardan görüldüğü üzere test grubu ve control grubu arasında fark olduğunu görüyoruz. 
# Peki gerçekten istatistiksel olarak anlamlı bir fark var mı?

#### Görev 3: Hipotez Testinin Gerçekleştirilmesi

Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz.

In [29]:
#H0: Normal dağılım varsayımı sağlanmaktadır.
# H1: Normal dağılım varsayımı sağlanmamaktadır.
#p-value < 0.05 ise H0 Reddedilir. 
# shapiro testini çağırıyoruz. grubun içindeki control olanları seçip purchase kolonuna getirecek. 
test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# Aşağıdan görüldüğü üzere p-value 0.05ten büyük yani h0 reddedilemez. 
# Normallik varsayımı sağlanmaktadır. 

Test Stat = 0.9773, p-value = 0.5891


In [33]:
# Yukardaki işlemin aynısını test grubu için de yapalım; 
# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1: Normal dağılım varsayımı sağlanmamaktadır.
#p-value < 0.05 ise H0 Reddedilir. 
test_stat,pvalue = shapiro(df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# Aşağıda görüldüğü üzere p-value 0.05ten büyül yani h0 reddedilemez. 
# Varyanslar homojendir. 
#Normallik varsayımı sağlandığını görüşmüş olduk. Sağlanmıyor olsaydı nonparametrik bir test yönteminr geçmemiz gerekirdi. 
# Şimdi varyans homojenliğine bakacağız. 

Test Stat = 0.9589, p-value = 0.1541


In [39]:
# Varyans Homojenliği :
# H0: Varyanslarhomojendir.
# H1: Varyanslarhomojen Değildir.
# p < 0.05 H0 RED
# Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediyoruz.

test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# p-value > 0.05 olduğu için h0 reddedilemez. Varyanslar homojendir. 

Test Stat = 2.6393, p-value = 0.1083


In [43]:
# Görüldüğü üzere varyanslar homojen ve normal bir dağılım var bu sebeple bağımsız iki örneklem t testi(parametrik test) yapacağız.
# H0: M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında ist. ol.anl.fark yoktur.)
# H1: M1 != M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında ist. ol.anl.fark vardır)
# p<0.05 HO RED
test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# p-value > 0.05 h0 reddedilemez. 
# m1 ve m2 arasında istatistiksel olarak anlamlı bir fark yoktur. 


Test Stat = -0.9416, p-value = 0.3493


        İlk önce iki gruba da normallik testi uyguladık. İki grubun da normal dağılımı(shapiro) uygun olduğu gözlemlendiği için ikinci varsayımımız olan varyans homojenliğini(levene) inceledik. Varyanslar da homojen olduğu için "Bağımsız iki örneklem t testi" uyguladık. Uygulama sonucunda da h0 > 0.05 gözlemlendi. Dolayısıyla h0'ı reddedemedik. İki örneklem arasında anlamlı bir fark yoktur dedik. 


        ** Satın alma anlamından anlamlı bir fark olmadığı için müşteri iki yöntemi de seçebilir. Fakat burada diğer istatistikler de önem arz edebilir. 